In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_5030/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)


    def keep_random_models(self, n_models=3):
        # Extract block part from 'model' column
        self.results_df['block'] = self.results_df['model'].apply(lambda x: '_'.join(x.split('_')[1:3]))
        
        # Group by 'block' and keep n_models randomly from each group
        def select_random_models(group):
            return group.sample(n=min(n_models, len(group)), random_state=1)
        
        # Apply the function and ensure grouping columns are excluded from the final DataFrame
        grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)
        self.results_df = grouped.reset_index(drop=True).drop(columns=['block'])

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.keep_random_models()
        self.results_df = self.results_df.sort_values(by="r2_sup", ascending=False, ignore_index=True)
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



/tmp/ipykernel_5030/3323087823.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)


,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_6_18_2,0.951051,0.685116,0.738809,0.935973,0.418344,2.105629,3.215324,0.636479,1.591332,0.646795,0.937301,0.652494,51.742903,120.366207,"Hidden Size=[4, 2], regularizer=0.02, learning..."
1,model_6_17_7,0.951213,0.682860,0.740882,0.934260,0.416960,2.120714,3.189815,0.666873,1.588740,0.645724,0.937508,0.651414,51.749531,120.372834,"Hidden Size=[4, 2], regularizer=0.02, learning..."
2,model_2_31_4,0.976343,0.658901,0.857410,0.956981,0.202183,2.280932,1.755325,0.251463,1.115477,0.449648,0.966705,0.453610,69.197163,159.779923,"Hidden Size=[8], regularizer=0.02, learning_ra..."
3,model_2_31_5,0.976532,0.658481,0.855711,0.954241,0.200569,2.283741,1.776231,0.267480,1.152675,0.447850,0.966971,0.451796,69.213190,159.795950,"Hidden Size=[8], regularizer=0.02, learning_ra..."
4,model_2_31_0,0.973402,0.658208,0.870552,0.967979,0.227322,2.285562,1.593535,0.187176,0.944771,0.476783,0.962565,0.480984,68.962773,159.545533,"Hidden Size=[8], regularizer=0.02, learning_ra..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590,model_3_33_1,0.988329,-17.493489,0.623949,0.987604,0.099750,123.666048,4.629290,0.065165,1.313305,0.315832,0.979530,0.318615,102.610174,237.111848,"Hidden Size=[12], regularizer=0.02, learning_r..."
591,model_3_39_2,0.988615,-17.649332,0.547828,0.985849,0.097299,124.708169,5.566349,0.153391,0.976064,0.311928,0.980033,0.314676,102.659935,237.161609,"Hidden Size=[12], regularizer=0.02, learning_r..."
592,model_3_35_6,0.987339,-17.941495,0.507968,0.932446,0.108205,126.661862,6.057041,0.292520,1.052430,0.328945,0.977795,0.331843,102.447461,236.949135,"Hidden Size=[12], regularizer=0.02, learning_r..."
593,model_3_39_6,0.988455,-21.225768,0.459230,0.978363,0.098673,148.623813,6.657020,0.234533,0.910317,0.314122,0.979751,0.316890,102.631890,237.133564,"Hidden Size=[12], regularizer=0.02, learning_r..."
